In [ ]:
from pymongo import MongoClient
import pymongo
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import pickle
from tmqr.settings import *
from tmqrfeed.contracts import OptionContract

In [ ]:
%load_ext Cython
%load_ext line_profiler
%load_ext memory_profiler

In [ ]:
instrument = 'TST'
market = 'T'
future = 'F-TST-H11-110322'
opt_expiration = '110322'

In [ ]:
options = ['C', 'P']
N = 60

In [ ]:
strikes = list(np.arange(50.0, 100.0, 1.0))

In [ ]:
options_tickers_list = []
for opt_type in options:
    for s in strikes:
        #US.C.F-CL-H11-110322.110121@89.0
        options_tickers_list.append("{0}.{1}.{2}.{3}@{4}".format(market, opt_type, future,  opt_expiration, s))        

In [ ]:
dates = [datetime(2011, 1, 2) + timedelta(days=x) for x in range(N)]

client = MongoClient(MONGO_CONNSTR)
db = client[MONGO_DB]

In [ ]:


for i, opt in enumerate(options_tickers_list):
    iv_arr = np.random.random(size=N)
    price_arr = np.random.random(size=N)
    to_exp = np.arange(N)
    
    df = pd.DataFrame({'iv': iv_arr, 'px': price_arr, 'toexp': to_exp}, index=dates)
    #
    # Writing bundled quotes
    # 
    db['quotes_options_eod_bundled'].create_index([('tckr', pymongo.ASCENDING)])
    db['quotes_options_eod_bundled'].create_index([('idx', pymongo.ASCENDING)])
    db['quotes_options_eod_bundled'].replace_one({'tckr': opt},
                                                 {'tckr': opt, 
                                                  'data': pickle.dumps(df),
                                                  'idx': i,
                                                 },
                                                 upsert=True
    )
    #
    # Writing single records
    # 
    db['quotes_options_eod_single'].create_index([('tckr', pymongo.ASCENDING), ('dt', pymongo.ASCENDING)])
    db['quotes_options_eod_single'].create_index([('idx', pymongo.ASCENDING)])
    for i, dt in enumerate(dates):
        """print({
            'tckr': opt,
            'dt': dt,
            'iv': iv_arr[i],
            'px': price_arr[i],
            'to_exp': to_exp[i]
        })
        """
        db['quotes_options_eod_single'].replace_one({'tckr': opt, 'dt': dt},
            {
                'tckr': opt,
                'dt': dt,
                'iv': iv_arr[i],
                'px': price_arr[i],
                'idx': i,
                #'to_exp': to_exp[i]
            }, upsert=True)
    
    
    
    

In [ ]:
class ChainSingle:
    def __init__(self, opt_tickers):
        client = MongoClient(MONGO_CONNSTR)
        self.db = client[MONGO_DB]
        self.tickers = opt_tickers
        
    def compose(self, date):
        result = db['quotes_options_eod_single'].find({'tckr': {'$in': self.tickers}, 'dt': date})
        #result = db['quotes_options_eod_single'].find({'idx': {'$in': list(range(len(self.tickers)))}, 'dt': date})
        return pd.DataFrame(list(result))
    
    def get_one(self, tckr, date):
        result = db['quotes_options_eod_single'].find({'tckr': tckr, 'dt': date})
        return pd.DataFrame(list(result))
        
class ChainBundled:
    def __init__(self, opt_tickers):
        client = MongoClient(MONGO_CONNSTR)
        self.db = client[MONGO_DB]
        self.tickers = opt_tickers
        self.panel = None
        self.one_df = None
        
    def compose(self, date):
        if self.panel is None:
            result = db['quotes_options_eod_bundled'].find({'tckr': {'$in': self.tickers}})  
            #result = db['quotes_options_eod_bundled'].find({'idx': {'$in': list(range(len(self.tickers)))}})  
            
            res_dict = {}
            for res in result:
                res_dict[res['tckr']] = pickle.loads(res['data'])

            self.panel = pd.Panel(res_dict)
        return self.panel.major_xs(date)
    
    def get_one(self, tckr, date):
        if self.one_df is None:
            result = db['quotes_options_eod_bundled'].find_one({'tckr': tckr})        
        
            self.one_df = pickle.loads(result['data'])
        return self.one_df.loc[date]
        
        
        

# Chaing composition benchmark

In [ ]:
#%%timeit 
chain1 = ChainSingle(options_tickers_list)
chain1.compose(datetime(2011, 1, 2))

In [ ]:
#%%timeit
chain2 = ChainBundled(options_tickers_list)
chain2.compose(datetime(2011, 1, 2))

In [ ]:
p = chain2.panel

In [ ]:
mem = 0
for i in p:
    mem += p.loc[i].memory_usage()
print(mem)


In [ ]:
%%timeit
p.loc['T.C.F-TST-H11-110322.110322@50.0'].at[datetime(2011, 1, 2), 'iv']

In [ ]:
%timeit chain1.compose()

In [ ]:
%timeit chain2.compose(datetime(2011, 1, 3))

In [ ]:
%%timeit 
chain1 = ChainSingle(options_tickers_list)
for i in range(20):
    chain1.compose(datetime(2011, 1, 2) + timedelta(days=i))

In [ ]:
%%timeit 
chain2 = ChainBundled(options_tickers_list)

for i in range(20):
    chain2.compose(datetime(2011, 1, 2) + timedelta(days=i))

# Single element fetching benchmark

In [ ]:
%%timeit 
chain1 = ChainSingle(options_tickers_list)
chain1.get_one('T.C.F-TST-H11-110322.110322@50.0', datetime(2011, 1, 2))

In [ ]:
%%timeit 
chain2 = ChainBundled(options_tickers_list)
chain2.get_one('T.C.F-TST-H11-110322.110322@50.0', datetime(2011, 1, 2))

In [ ]:
%%timeit 
chain1 = ChainSingle(options_tickers_list)
for i in range(20):
    chain1.get_one('T.C.F-TST-H11-110322.110322@50.0', datetime(2011, 1, 2) + timedelta(days=i))

In [ ]:
%%timeit 
chain2 = ChainBundled(options_tickers_list)

for i in range(20):
    chain2.get_one('T.C.F-TST-H11-110322.110322@50.0', datetime(2011, 1, 2) + timedelta(days=i))

In [ ]:
chain2 = ChainBundled(options_tickers_list)
chain2.get_one('T.C.F-TST-H11-110322.110322@50.0', datetime(2011, 1, 2))

# Option chains benchmarks

In [ ]:
#%%timeit 
cursor = db['asset_index'].aggregate([
            {'$match': {
                'underlying': 'US.F.ES.H11.110318',
                'type': {'$in': ['P', 'C']},                
            }},
            
            {'$sort': {'strike': 1}},
            
            {'$project': {'tckr': 1, 'exp': 1, 'strike': 1, 'type': 1}
            },
            
            {'$group': {
                '_id': {'date': '$exp'},
                'chain': {'$push': '$$ROOT'},
            }
            },
            {'$sort': {"_id.date": 1}}          
])
chain_list = list(cursor)

In [ ]:
%timeit [OptionContract(opt['tckr']) for opt in chain_list[0]['chain']]

In [ ]:
chain_list

In [ ]:
%timeit -n 1 'T.C.F-TST-H11-110322.110322@50.0'.__hash__()

In [ ]:
%timeit float('nan')

In [ ]:
%lprun -f OptionContract.__init__ [OptionContract(opt['tckr']) for opt in chain_list[0]['chain']]

In [ ]:
#%%timeit 
from collections import OrderedDict

chain_result = OrderedDict()

strike_count = 0

for exp in chain_list:
    options = chain_result.setdefault(exp['_id']['date'], OrderedDict())
    
    prev_strike = 0.0
    chain = exp['chain']
    for i, strike_rec in enumerate(chain):
        strike_count += 1
        strike = strike_rec['strike']
        if i == 0:
            continue
                
        if strike == chain[i-1]['strike']:
            # We have put call pair             
            if strike_rec['type'] == 'C':
                call_idx = i
                put_idx = i-1
            else:
                call_idx = i-1
                put_idx = i
            
            options[strike] = (chain[call_idx]['tckr'], chain[put_idx]['tckr'])    
    

In [ ]:
strike_count

In [ ]:
with open("chain_list_es.pkl", 'wb') as f:
    pickle.dump(chain_list, f)

In [ ]:
[x['_id']['date'] for x in chain_list]

In [ ]:
%timeit chain2.panel.ix['T.C.F-TST-H11-110322.110322@50.0']

In [ ]:
df = chain2.panel.ix['T.C.F-TST-H11-110322.110322@50.0']

In [ ]:
%timeit df.at[datetime(2011,1, 2), 'iv']

In [ ]:
[x for x in [1, 2, 3]]